##IMPORT LIBRARY

In [ ]:
pip install transformers

In [ ]:
#general purpose packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from tensorflow import keras
import tensorflow as tf

#metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

## LOAD DATASET
Dataset is scrapped from https://putusan3.mahkamahagung.go.id/pengadilan/profil/pengadilan/pn-negara

In [ ]:
df = pd.read_excel('/content/data_siap.xlsx')

df.head()


,laporan,label,story
0,hadapan pemuka Agama Hindu yang bernama Mangku...,perdata,alami cerai mang i komang don sebab anakanak g...
1,ngkan secara adat Bali dan agama Hindu pada ta...,perdata,alami cerai pasang adat bal agama hindu lurah ...
2,Menyatakan Tergugat telah dipanggil dengan pat...,perdata,alami cerai gugat tanggal mei kabupaten jembra...
3,Menyatakan Tergugat telah dipanggil dengan pat...,perdata,alami gugat hadir panggil adil gugat gugat kab...
4,Menyatakan Tergugat telah dipanggil dengan pat...,perdata,alami gugat hadir panggil adil gugat menang gu...


In [ ]:
print(df['label'].value_counts())

label
perdata          198
pidana           191
perdata_agama    132
Name: count, dtype: int64


In [ ]:
df = df[['story','label']]

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
token_lens = []

for txt in df['story'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

max_len=np.max(token_lens)

print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 78


## BALANCING DATA USING RANDOM OVER SAMPLING
To avoid models more likely to study certain classes (unbalanced data). Then Data Balancing is needed, one of which is by using Random Over Sampling.

In [ ]:
ros = RandomOverSampler()
x, y = ros.fit_resample(np.array(df['story']).reshape(-1, 1), np.array(df['label']).reshape(-1, 1));
os = pd.DataFrame(list(zip([x[0] for x in x], y)), columns = ['story', 'label']);

os['label'].value_counts()

label
perdata          198
pidana           198
perdata_agama    198
Name: count, dtype: int64

In [ ]:
x, y = os['story'].values, os['label'].values
x_train, x_valid, y_train, y_valid = train_test_split(x, y, stratify = y, random_state = 40, test_size = 0.2, shuffle = True)

In [ ]:
# Jumlah data latih
num_train = len(x_train)
print("Jumlah data latih:", num_train)

# Jumlah data uji
num_test = len(x_valid)
print("Jumlah data uji:", num_test)

Jumlah data latih: 475
Jumlah data uji: 119


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
label_counts = dict(zip(unique, counts))
print(label_counts)

{'perdata': 159, 'perdata_agama': 158, 'pidana': 158}


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)

## TOKENIZER USING BERT-BASE-UNCASED
The tokenized words are converted into a vector where BERT-base-uncased has a glossary of 30,522 words.

In [ ]:
MAX_LEN=128

def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize(x_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(x_valid, MAX_LEN)

## BUILDING MODEL BERT
The BERT model is built with a Deep Learning structure, namely Keras. And the output layer is adjusted to the class to be classified.

In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    embeddings = bert_model([input_ids,attention_masks])[1]
    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)
    model_out = tf.keras.models.Model(inputs=[input_ids, attention_masks], outputs=output)
    model_out.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    return model_out

In [ ]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_1[0][0]',             
 )                           ngAndCrossAttentions(last_   40         'input_2[0][0]']             
                             hidden_state=(None, 128, 7                                           
                             68),                                                             

In [ ]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=5, batch_size=42)

Epoch 1/5
12/12 [==============================] - 101s 2s/step - loss: 0.8872 - accuracy: 0.5705 - val_loss: 0.5049 - val_accuracy: 0.7395
Epoch 2/5
12/12 [==============================] - 12s 1s/step - loss: 0.4981 - accuracy: 0.7221 - val_loss: 0.4344 - val_accuracy: 0.7647
Epoch 3/5
12/12 [==============================] - 12s 1000ms/step - loss: 0.4309 - accuracy: 0.7895 - val_loss: 0.5228 - val_accuracy: 0.7479
Epoch 4/5
12/12 [==============================] - 12s 1s/step - loss: 0.4079 - accuracy: 0.7895 - val_loss: 0.5209 - val_accuracy: 0.8487
Epoch 5/5
12/12 [==============================] - 12s 1s/step - loss: 0.2969 - accuracy: 0.8611 - val_loss: 0.4632 - val_accuracy: 0.8235


## ACCURACY MODEL BERT

In [ ]:
train_loss, train_accuracy = model.evaluate([train_input_ids,train_attention_masks], y_train)

print(f'Train Loss: {train_loss}')
print(f'Train Accuracy: {train_accuracy}')

15/15 [==============================] - 4s 270ms/step - loss: 0.2634 - accuracy: 0.9032
Train Loss: 0.26343223452568054
Train Accuracy: 0.9031578898429871


In [ ]:
val_loss, val_accuracy = model.evaluate([val_input_ids, val_attention_masks], y_valid)

print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

4/4 [==============================] - 1s 250ms/step - loss: 0.4632 - accuracy: 0.8235
Validation Loss: 0.46324992179870605
Validation Accuracy: 0.8235294222831726
